# Anatomy of ring and motor-related neurons in the hindbrain

In [ ]:
%matplotlib widget
import numpy as np
import pandas as pd

import lotr.plotting as pltltr
from lotr import A_FISH, LotrExperiment, dataset_folders
from lotr.utils import crop, resample_matrix, zscore

COLS = pltltr.COLS
from matplotlib import pyplot as plt
from tqdm import tqdm

## Localization of ring neurons
First, let's have a look at the positions of ring neurons. We will also load information about the motor regressors, as later we will also plot the localizations of the motor-selective neurons.

In [ ]:
Z_SCATTER_FACTOR = 10  # add some random dispersion in z to make all rois visible
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    cent_coords = exp.morphed_coords_um
    data_dict = dict()
    for i, lab in enumerate(["s_i", "l_r", "a_p"]):
        data_dict.update({lab: cent_coords[:, i]})
    data_dict["s_i"] = (
        data_dict["s_i"] + np.random.rand(len(data_dict["a_p"])) * Z_SCATTER_FACTOR
    )
    data_dict["fid"] = path.name
    data_dict["hdn"] = ~exp.nonhdn_indexes

    data_df.append(pd.concat([pd.DataFrame(data_dict), exp.motor_regressors], axis=1))

data_df = pd.concat(data_df, ignore_index=1)

## Motor selectivity

Now we look at how much motor selective are neurons

In [ ]:
lcol, rcol = COLS["phase"](0.6), COLS["phase"](0.05)
s = 10

f, ax = plt.subplots(figsize=(3.5, 4))
ax.scatter(
    data_df["l_r"],
    data_df["a_p"],
    lw=0,
    s=s,
    alpha=0.05,
    color=(0.3,) * 3,
    rasterized=True,
)
thr = 0.7
for reg, regnot, c in zip(["right_1", "left_1"], ["left_1", "right_1"], [rcol, lcol]):
    motor_sel = (data_df[reg] > thr) & (data_df[regnot] < thr)
    lab = reg.split("_")[0]
    ax.scatter(
        data_df["l_r"][motor_sel],
        data_df["a_p"][motor_sel],
        # c=data_df[reg][motor_sel],
        label=f"mot. {lab} > {thr}",
        color=c,
        # cmap=cmap,
        lw=0,
        vmin=0.0,
        vmax=1,
        s=s,
        alpha=0.2,
        rasterized=True,
    )
loc = [-110, -138]
ax.axis("equal")
ax.set_title("Horizontal view")
l = ax.legend(loc=2, bbox_to_anchor=(0.8, -0.2))
for lh in l.legendHandles:
    lh.set_alpha(1)
pltltr.add_anatomy_scalebar(ax, pos=loc, length=30, fontsize=6)
pltltr.savefig("horview_motorneurons")

ax.scatter(
    data_df["l_r"][data_df["hdn"]],
    data_df["a_p"][data_df["hdn"]],
    ec=COLS["ring"],
    fc="none",
    lw=0.1,
    s=s,
    alpha=0.5,
    label="r1π ROIs",
    rasterized=True,
)
l = ax.legend(loc=2, bbox_to_anchor=(0.7, 0.2))
for lh in l.legendHandles:
    lh.set_alpha(1)
pltltr.savefig("horview_motorneurons_andring")

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(7, 2.5))
s = 8
sel = data_df["hdn"]
planes = "horizontal", "sagittal", "frontal"
locs = [[-110, -138], [-52, -156], [-105, -71]]
for i, coords in enumerate(
    [
        [data_df["l_r"], data_df["a_p"]],
        [data_df["s_i"], data_df["a_p"]],
        [data_df["l_r"], data_df["s_i"]],
    ]
):
    axs[i].scatter(
        coords[0], coords[1], lw=0, s=s, alpha=0.03, color=(0.3,) * 3, rasterized=True
    )
    for reg, regnot, c in zip(
        ["right_1", "left_1"],
        ["left_1", "right_1"],
        [rcol, lcol],
    ):
        motor_sel = (data_df[reg] > thr) & (data_df[regnot] < thr)
        lab = reg.split("_")[0]
        axs[i].scatter(
            coords[0][motor_sel],
            coords[1][motor_sel],
            # c=data_df[reg][motor_sel],
            label=f"mot. {lab} > {thr}",
            color=c,
            # cmap=cmap,
            lw=0,
            vmin=0.0,
            vmax=1,
            s=s,
            alpha=0.2,
            rasterized=True,
        )

    axs[i].axis("equal")
    axs[i].axis("off")
    axs[i].set_title(planes[i].capitalize() + " view")

    pltltr.add_anatomy_scalebar(
        axs[i], plane=planes[i], pos=locs[i], length=30, fontsize=6
    )

l = axs[2].legend(loc=2, bbox_to_anchor=(0.5, 0.3))
for lh in l.legendHandles:
    lh.set_alpha(1)
pltltr.savefig("allviews_allneurons_motor", folder="S10")

for i, coords in enumerate(
    [
        [data_df["l_r"], data_df["a_p"]],
        [data_df["s_i"], data_df["a_p"]],
        [data_df["l_r"], data_df["s_i"]],
    ]
):
    axs[i].scatter(
        coords[0][data_df["hdn"]],
        coords[1][data_df["hdn"]],
        s=s,
        color=COLS["ring"],
        ec=COLS["ring"],
        lw=0.1,
        alpha=0.1,
        label="r1π ROIs",
        rasterized=True,
    )
l = axs[2].legend(loc=2, bbox_to_anchor=(0.5, 0.3))
for lh in l.legendHandles:
    lh.set_alpha(1)
pltltr.savefig("allviews_allneurons_motor_ring", folder="S10")

## Ring neurons motor regressors scatter

In [ ]:
f, ax = plt.subplots(figsize=(2.5, 2.5), gridspec_kw=dict(left=0.2, bottom=0.2))
s = 5
plt.scatter(
    data_df["left_1"],
    data_df["right_1"],
    alpha=0.1,
    lw=0,
    color=".3",
    s=s,
    label="All ROIs",
    rasterized=True,
)
plt.scatter(
    data_df.loc[data_df["hdn"], "left_1"],
    data_df.loc[data_df["hdn"], "right_1"],
    alpha=0.6,
    lw=0,
    color=COLS["ring"],
    label="r1π ROIs",
    s=s,
    rasterized=True,
)
ax.axis("equal")
ticks = [-0.5, 0, 0.5, 1]
lims = ticks[0] - 0.1, ticks[-1] + 0.1
ax.set(
    xlabel=("Corr. with left swims"),
    ylabel=("Corr. with right swims"),
    xticks=ticks,
    yticks=ticks,
    xlim=lims,
    ylim=lims,
)
plt.legend(
    loc=2, bbox_to_anchor=(0.6, 0.25), labelcolor=(".5", COLS["ring"]), handlelength=0
)
pltltr.despine(ax)
plt.show()
pltltr.savefig("left_right_swimcorrs", folder="S10")

## Plot example regressors

In [ ]:
from lotr.behavior import create_motor_regressors

In [ ]:
# Example traces from a fish
exp = LotrExperiment(A_FISH)
reg_dict = create_motor_regressors(exp.n_pts, exp.bouts_df, exp.fs, min_bias=0.05)
cent_coords = exp.morphed_coords_um

In [ ]:
f, axs = plt.subplots(
    3,
    2,
    figsize=(5, 2.5),
    gridspec_kw=dict(hspace=0.1, right=0.95, wspace=0.05, width_ratios=(1, 0.15)),
)
t_lims_s = 5, 820
t_slice = slice(*[t * exp.fs for t in t_lims_s])
t_arr = np.arange(t_lims_s[1] * exp.fs - t_lims_s[0] * exp.fs) / exp.fs

sel_beh = (exp.behavior_log["t"] < t_lims_s[1]) & (exp.behavior_log["t"] > t_lims_s[0])
toplot_beh = exp.behavior_log[sel_beh].copy()
toplot_beh["t"] = toplot_beh["t"] - toplot_beh["t"].values[0]

for i, (reg, regnot, c, ax, ax_sc) in enumerate(
    zip(
        ["right_1", "left_1"],
        ["left_1", "right_1"],
        [rcol, lcol],
        axs[1:, 0],
        axs[1:, 1],
    )
):
    # sel_traces = exp.traces[:, motor_sel]
    # sel_reg = exp.motor_regressors[motor_sel]
    ax.fill(t_arr, reg_dict[reg][t_slice] - 1, fc=pltltr.shift_lum(c, 0.2))
    mot_sel = exp.motor_regressors.loc[
        (exp.motor_regressors[reg] > thr) & (exp.motor_regressors[regnot] < thr), reg
    ]
    cidx = mot_sel.sort_values().index[-2 - 4 * i]
    ax.plot(t_arr, zscore(exp.traces[t_slice, cidx]), c=c, lw=1)

    ax_sc.scatter(cent_coords[:, 1], cent_coords[:, 2], s=5, color=".5", alpha=0.1)
    ax_sc.scatter(cent_coords[cidx, 1], cent_coords[cidx, 2], s=5, color=c)
    ax_sc.axis("equal")

    ax.text(
        0,
        -1.2,
        reg.split("_")[0] + f" swim regressor (corr: {round(mot_sel[cidx], 2):0.2f})",
        c=c,
        va="top",
    )
    ax.set(ylim=(-2, 4))

axs[0, 0].plot(
    toplot_beh["t"], zscore(toplot_beh["tail_sum"]) / 10, c=".6", lw=1, rasterized=True
)
axs[0, 0].text(0, 1, "Tail angle", c=".6", va="top")
pltltr.add_anatomy_scalebar(axs[2, 1], pos=(-80, -150), c=".3", fontsize=6)

[pltltr.despine(ax, "all") for ax in axs.flatten()]

pltltr.savefig("example_motor_neurons", folder="S10")

## Time constants

In [ ]:
exp_cell_data = data_df[data_df["fid"] == path.name]

In [ ]:
sel_vects = []
for reg, regnot, c in zip(["right_1", "left_1"], ["left_1", "right_1"], [rcol, lcol]):
    sel_vects.append((exp_cell_data[reg] > thr) & (exp_cell_data[regnot] < thr))
motor_sel = np.array(sel_vects).any(0)

In [ ]:
PRE_BOUT_WND_S = 5
POST_BOUT_WND_S = 30
FS_TO_MATCH = 5

cropped_traces_lf = []
cropped_traces_rt = []
cells_sel_df = []

time_arr = (
    np.arange(1, ((PRE_BOUT_WND_S + POST_BOUT_WND_S) * FS_TO_MATCH) + 1) / FS_TO_MATCH
    - PRE_BOUT_WND_S
)

for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)

    exp_cell_data = data_df[data_df["fid"] == path.name]

    sel_vects = []
    for reg, regnot, c in zip(
        ["right_1", "left_1"], ["left_1", "right_1"], [rcol, lcol]
    ):
        sel_vects.append((exp_cell_data[reg] > thr) & (exp_cell_data[regnot] < thr))
    motor_sel = np.array(sel_vects).any(0)

    traces = exp.raw_traces[:, motor_sel]
    traces = (traces - np.nanmean(traces, 0)) / np.nanstd(traces, 0)
    # Crop around events:
    cropped = crop(
        exp.raw_traces[:, motor_sel],
        exp.bouts_df["idx_imaging"],
        pre_int=PRE_BOUT_WND_S * exp.fs,
        post_int=POST_BOUT_WND_S * exp.fs,
    )

    # Subtract baseline:
    cropped = cropped - np.nanmean(cropped[: PRE_BOUT_WND_S * exp.fs, :], 0)

    # Interpolate if necessary:
    if exp.fs != FS_TO_MATCH:
        # This interpolation code would be needed for a single fish but is sometimes giving
        # troubles b/c of a numba function. If you really care about that one fish, you can try and uncomment - it won't change much.

        # fish_time_arr = np.arange(1, cropped.shape[0] + 1) / exp.fs - PRE_BOUT_WND_S
        # cropped = resample_matrix(time_arr, fish_time_arr, cropped)
        pass
    else:
        cropped_traces_lf.append(
            np.nanmean(cropped[:, exp.bouts_df["direction"] == "lf", :], 1)
        )
        cropped_traces_rt.append(
            np.nanmean(cropped[:, exp.bouts_df["direction"] == "rt", :], 1)
        )
        cells_sel_df.append(exp_cell_data[motor_sel])

cropped_traces_lf = np.concatenate(cropped_traces_lf, axis=1)
cropped_traces_rt = np.concatenate(cropped_traces_rt, axis=1)
cells_sel_df = pd.concat(cells_sel_df)

In [ ]:
parameter_rt = cropped_traces_rt[50:60, :].mean(0) / cropped_traces_rt[28:32, :].mean(0)
parameter_lf = cropped_traces_lf[50:60, :].mean(0) / cropped_traces_lf[28:32, :].mean(0)

In [ ]:
cells_sel_df["parameter_rt"] = parameter_rt
cells_sel_df["parameter_lf"] = parameter_lf

In [ ]:
for reg, regnot, par in zip(
    ["right_1", "left_1"], ["left_1", "right_1"], ["parameter_rt", "parameter_lf"]
):
    sel = (cells_sel_df[reg] > thr) & (cells_sel_df[regnot] < thr)
    f, ax = plt.subplots()
    s = 20
    ax.scatter(
        cells_sel_df.loc[sel, "l_r"],
        cells_sel_df.loc[sel, "a_p"],
        # c=data_df[reg][motor_sel],
        label=f"mot. {lab} > {thr}",
        c=cells_sel_df.loc[sel, par],
        lw=0,
        vmax=1,
        vmin=0.1,
        s=s,
        alpha=1,
    )